# **<h1><center> Header and Production Data Cleaning </center></h1>**

# Purpose
- Header Data:
    - Cast Dates to dates
    - Get top perf, bottom perf, and total perf length
    - Clean operator name
    - Get location data (township/range/block)
    - Get 12m/24m/36m Production
- Production Data
    - Caset Dates
    - Calculate WOR & GOR
    - Normalize production for perforated length

## Import Packages

In [34]:
import pandas as pd
import utm
import sqlite3
import random
import numpy as np
from statistics import mean 
import plotly.express as px
pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_columns', None)
import plotly.graph_objects as go
pd.set_option("display.max_rows",1000); pd.options.display.precision = 1
import warnings
warnings.filterwarnings("ignore")

### Connect to SQlite3 DataBase

In [35]:
%load_ext sql
%sql sqlite:////Users/john.odonnell/Python/Web_Scraping/NDIC/Well_DataBase.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### View Tables

In [36]:
%%sql 

SELECT 
    name 
FROM sqlite_master 
WHERE type ='table';

 * sqlite:////Users/john.odonnell/Python/Web_Scraping/NDIC/Well_DataBase.db
Done.


name
prod_table
header_table
header_table_clean
prod_table_clean


# <center> --------------------            Header Data             -------------------- </center>

## Import Header data

In [37]:
query = %sql SELECT * FROM header_table
df_header = query.DataFrame()
df_header.set_index('UWI',inplace=True)

 * sqlite:////Users/john.odonnell/Python/Web_Scraping/NDIC/Well_DataBase.db
Done.


## Feature Manipulation and Cleaning


In [38]:
# Cast Dates
df_header[['Completion_Date','Spud_Date']] = df_header[['Completion_Date','Spud_Date']].apply(pd.to_datetime, errors='coerce')
df_header['Vintage_Year'] = pd.DatetimeIndex(df_header['Completion_Date']).year

# Get perf info
df_header[['Top_Perf','Bottom_Perf']] = df_header['Perf_Interval'].str.split('-',n=1,expand=True)
df_header['Top_Perf'] = df_header['Top_Perf'].str.replace('[^0-9]','').replace('',np.nan)
df_header['Bottom_Perf'] = df_header['Bottom_Perf'].str.replace('[^0-9]','').replace('',np.nan)

# Clean operator name
df_header[['Operator','delete']] = df_header['Current_Operator'].str.split(" ",n=1,expand=True).replace(',','')

# Cast and clean
df_header[['Top_Perf','Bottom_Perf']] = df_header[['Top_Perf','Bottom_Perf']].astype(float)
df_header = df_header[df_header['Top_Perf'].between(5000,13000)]

# Get perforated length
df_header['Perforated_Length'] = df_header['Bottom_Perf'] - df_header['Top_Perf']

# Get the Township/Range/Section Identifiers
df_header[['Direction','TRS']] = df_header['Location'].str.split(" ",n=1,expand=True)
df_header[['Section','Township','Range']] = df_header['TRS'].str.split("-",n=2,expand=True)
df_header['Block'] = (df_header['Township'] + df_header['Range']).astype(int)
df_header.drop(columns=['delete'],inplace=True)

## Write new cleaned table to database

In [39]:
# Create connection
cnx = sqlite3.connect(r"/Users/john.odonnell/Python/Web_Scraping/NDIC/Well_DataBase.db")

# Write to DataBase
df_header.to_sql("header_table_clean",cnx, if_exists='replace')

# <center> ------------------------ Production Data -------------------------------- </center>

## Import Production data

In [40]:
# Create Query
query = %sql SELECT UWI,Date,Days,Oil,Water,Gas FROM prod_table

# Read query to DataFrame, set index
df_production = query.DataFrame()
df_production.set_index('UWI',inplace=True)

 * sqlite:////Users/john.odonnell/Python/Web_Scraping/NDIC/Well_DataBase.db
Done.


## Prepare Production Data
- Cast `Date` to datetime
- Sort data by date
- Calculate total fluid, WOR, GOR

In [41]:
# Cast Dates
df_production['Date'] = pd.to_datetime(df_production['Date'], infer_datetime_format=True)

# Sort
df_production.sort_values('Date',inplace=True)

# Calculate total fluid, WOR, and GOR
df_production['Fluid'] = df_production['Oil'] + df_production['Water']
df_production['WOR'] = round(df_production['Water'] / df_production['Oil'],3)
df_production['GOR'] = round(df_production['Gas'] / df_production['Oil'],3)

# View Results
print(df_production.shape)
df_production.describe().transpose()

(1250870, 8)


,count,mean,std,min,25%,50%,75%,max
Days,1235950.000,26.041,8.866,0.000,27.000,30.000,31.000,31.000
Oil,1235950.000,3026.708,4615.607,-1.000,761.000,1588.000,3269.000,136924.000
Water,1235950.000,2805.385,5373.506,-53.000,498.000,1297.000,2883.000,650679.000
Gas,1235950.000,4938.208,8549.141,0.000,928.000,2275.000,4986.000,200369.000
Fluid,1235950.000,5832.093,9095.762,-53.000,1575.000,3113.000,6228.000,652381.000
WOR,1166255.000,NaN,NaN,-inf,0.418,0.845,1.525,inf
GOR,1168494.000,inf,NaN,0.000,0.872,1.374,2.232,inf


### Normalize for length

In [42]:
# Bring in length from headers dataframe
df_production = df_production.merge(df_header[['Perforated_Length']], left_index= True, right_index= True)

# Loop through streams, normlalize
for stream in ['Oil','Water','Gas','Fluid']:
    df_production[f'{stream}_nrm'] = (df_production[stream]/df_production['Perforated_Length'])*10000

# View Results
df_production.describe()

,Days,Oil,Water,Gas,Fluid,WOR,GOR,Perforated_Length,Oil_nrm,Water_nrm,Gas_nrm,Fluid_nrm
count,1229920.000,1229920.000,1229920.000,1229920.000,1229920.000,1160710.000,1162935.000,1244619.000,1229805.000,1229805.000,1229805.000,1229805.000
mean,26.047,3012.728,2790.087,4905.585,5802.815,NaN,inf,8862.385,3446.720,3026.502,5532.725,6473.222
std,8.859,4587.527,5348.731,8488.216,9042.522,NaN,NaN,1975.082,9674.426,9508.809,25835.284,17991.182
min,0.000,-1.000,-53.000,0.000,-53.000,-inf,0.000,-10170.000,-1517842.105,-2614736.842,-2426842.105,-4132578.947
25%,27.000,760.000,497.000,926.000,1573.000,0.417,0.871,8979.000,891.944,602.841,1118.068,1869.980
50%,30.000,1585.000,1294.000,2267.000,3106.000,0.844,1.372,9480.000,1817.778,1455.943,2608.696,3551.688
75%,31.000,3259.000,2871.000,4961.000,6206.000,1.524,2.229,9848.000,3748.983,3155.058,5668.003,6982.564
max,31.000,136924.000,650679.000,200369.000,652381.000,inf,inf,17502.000,3446710.526,1720877.193,7422894.737,5155263.158


## Write Cleaned Table to DataFrame

In [43]:
# Create Connection
cnx = sqlite3.connect(r"/Users/john.odonnell/Python/Web_Scraping/NDIC/Well_DataBase.db")

# Write to DataBase
df_production[['Date','Days','Oil_nrm','Water_nrm','Gas_nrm','Fluid_nrm','WOR','GOR']].to_sql("prod_table_clean", cnx, if_exists='replace')

# <center> --------------------            Add Production to Header Data             -------------------- </center>

## Calulate production

In [44]:
# For months 12, 24, and 36
for m in range(12,37,12):
    # For each well in the DF
    for uwi in df_header.index:
        try:
            cum_days = df_production.loc[uwi,'Days'].cumsum()
            if m * 30.4 < max(cum_days):
                for stream in ['Oil_nrm','WOR']:
                    if stream in ['WOR']:
                        ratio_prod_list = df_production.loc[uwi,stream]
                        df_header.loc[uwi,f'{m}m_{stream}'] = round(np.interp( m * 30.4 , cum_days , ratio_prod_list ),3)

                    else:
                        cum_prod = df_production.loc[uwi,f'{stream}'].cumsum()
                        df_header.loc[uwi,f'{m}m_{stream}_cum'] = int(np.interp( m * 30.4 , cum_days , cum_prod ) )
        except: 
            print(uwi)
    print(m) 
df_header.iloc[:,-6:].describe().transpose()

33061038420000
33025030560000
12
33061038420000
33025030560000
24
33061038420000
36


,count,mean,std,min,25%,50%,75%,max
12m_Oil_nrm_cum,5.000,208151.400,136443.584,65555.000,74971.000,228739.000,294442.000,377050.000
12m_WOR,5.000,0.005,0.008,0.001,0.001,0.001,0.001,0.020
24m_Oil_nrm_cum,5.000,333645.200,220015.559,105933.000,120764.000,347325.000,496400.000,597804.000
24m_WOR,5.000,0.003,0.003,0.000,0.000,0.002,0.003,0.008
36m_Oil_nrm_cum,5.000,408722.400,261759.455,139222.000,157022.000,409105.000,634042.000,704221.000
36m_WOR,5.000,0.007,0.009,0.000,0.000,0.004,0.008,0.022


## Write new cleaned table to database

In [45]:
# Create connection
cnx = sqlite3.connect(r"/Users/john.odonnell/Python/Web_Scraping/NDIC/Well_DataBase.db")

# Write to DataBase
df_header.to_sql("header_table_clean",cnx, if_exists='replace')